# Anàlisi Exploratòria de Dades (EDA) - Dataset de Demència

Aquest notebook conté l'anàlisi exploratòria del dataset de demència, seguint les tasques especificades:

1. Càrrega i inspecció inicial
2. Distribució de la variable objectiu
3. Estadístiques univariants
4. Visualitzacions
5. Valors anòmals o absents
6. Anàlisi bivariada i correlacions
7. Resum de descobertes

In [ ]:
# Importació de llibreries necessàries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# Configuració de visualitzacions
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuració per mostrar totes les columnes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Càrrega i Inspecció Inicial

Carreguem el dataset i examinem la seva estructura bàsica.

In [ ]:
# Càrrega del dataset
# NOTA: Ajusta el path segons la ubicació del teu dataset
df = pd.read_csv('../data/dementia_dataset.csv')

print("Dataset carregat correctament!")
print(f"Dimensions del dataset: {df.shape[0]} files × {df.shape[1]} columnes")

In [ ]:
# Primeres files del dataset
print("Primeres 5 files del dataset:")
df.head()

In [ ]:
# Informació general del dataset
print("Informació general del dataset:")
df.info()

In [ ]:
# Tipus de dades per columna
print("\nTipus de dades:")
print(df.dtypes)

In [ ]:
# Noms de les columnes
print("\nColumnes del dataset:")
print(df.columns.tolist())

## 2. Distribució de la Variable Objectiu

Analitzem la distribució de casos amb i sense demència per entendre l'equilibri de classes.

In [ ]:
# Assumim que la variable objectiu es diu 'dementia' o similar
# Ajusta el nom de la columna segons el teu dataset
target_col = 'dementia'  # Canvia això si és necessari

# Recompte de casos
print("Distribució de la variable objectiu:")
print("="*50)
target_counts = df[target_col].value_counts()
print(target_counts)
print("\nPercentatges:")
target_percentages = df[target_col].value_counts(normalize=True) * 100
print(target_percentages.round(2))

In [ ]:
# Visualització de la distribució
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gràfic de barres
target_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribució de Casos: Demència vs No Demència', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Classe', fontsize=12)
axes[0].set_ylabel('Nombre de casos', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Afegir valors sobre les barres
for i, v in enumerate(target_counts):
    axes[0].text(i, v + 5, str(v), ha='center', va='bottom', fontweight='bold')

# Gràfic de pastís
colors = ['#2ecc71', '#e74c3c']
axes[1].pie(target_counts, labels=target_counts.index, autopct='%1.1f%%', 
            startangle=90, colors=colors, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Proporció de Classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Càlcul del desequilibri de classes
imbalance_ratio = target_counts.max() / target_counts.min()
print(f"\nRàtio de desequilibri de classes: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 1.5:
    print("⚠️ ATENCIÓ: El dataset presenta desequilibri de classes. Considera tècniques de balanceig.")
else:
    print("✓ El dataset està relativament equilibrat.")

## 3. Estadístiques Univariants

Analitzem les característiques individuals de cada variable.

In [ ]:
# Separar variables numèriques i categòriques
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Variables numèriques ({len(numeric_cols)}): {numeric_cols}")
print(f"\nVariables categòriques ({len(categorical_cols)}): {categorical_cols}")

### 3.1 Estadístiques de Variables Numèriques

In [ ]:
# Estadístiques descriptives per variables numèriques
print("Estadístiques descriptives de variables numèriques:")
print("="*80)
df[numeric_cols].describe().round(2)

In [ ]:
# Estadístiques addicionals (asimetria i curtosi)
print("\nAsimetria (Skewness) i Curtosi (Kurtosis):")
print("="*80)
stats_df = pd.DataFrame({
    'Asimetria': df[numeric_cols].skew(),
    'Curtosi': df[numeric_cols].kurtosis()
}).round(2)
print(stats_df)

### 3.2 Freqüències de Variables Categòriques

In [ ]:
# Freqüències per cada variable categòrica
for col in categorical_cols:
    print(f"\nDistribució de '{col}':")
    print("="*50)
    freq_table = pd.DataFrame({
        'Freqüència': df[col].value_counts(),
        'Percentatge': df[col].value_counts(normalize=True) * 100
    }).round(2)
    print(freq_table)
    print()

## 4. Visualitzacions

Creem visualitzacions per entendre millor les distribucions i diferències entre grups.

### 4.1 Histogrames de Variables Numèriques

In [ ]:
# Histogrames per totes les variables numèriques
n_cols = 3
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() if n_rows > 1 else [axes]

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribució de {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Freqüència')
    axes[idx].grid(True, alpha=0.3)

# Amagar eixos sobrants
for idx in range(len(numeric_cols), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 4.2 Box Plots per Grup (Demència vs No Demència)

In [ ]:
# Box plots comparatius
# Excloure la variable objectiu de les variables numèriques
numeric_features = [col for col in numeric_cols if col != target_col]

n_cols = 3
n_rows = (len(numeric_features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() if n_rows > 1 else [axes]

for idx, col in enumerate(numeric_features):
    sns.boxplot(data=df, x=target_col, y=col, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{col} per Grup', fontweight='bold')
    axes[idx].set_xlabel('Demència')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

# Amagar eixos sobrants
for idx in range(len(numeric_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 4.3 Violin Plots per Variables Clau

In [ ]:
# Violin plots per variables clau (ajusta segons el teu dataset)
# Exemple amb variables típiques d'un dataset de demència
key_vars = ['age', 'MMSE', 'brain_volume']  # Ajusta aquests noms segons el teu dataset

# Filtrar només les variables que existeixen
key_vars_available = [var for var in key_vars if var in df.columns]

if key_vars_available:
    fig, axes = plt.subplots(1, len(key_vars_available), figsize=(6*len(key_vars_available), 5))
    if len(key_vars_available) == 1:
        axes = [axes]
    
    for idx, var in enumerate(key_vars_available):
        sns.violinplot(data=df, x=target_col, y=var, ax=axes[idx], palette='muted')
        axes[idx].set_title(f'Distribució de {var} per Grup', fontweight='bold', fontsize=12)
        axes[idx].set_xlabel('Demència', fontsize=11)
        axes[idx].set_ylabel(var, fontsize=11)
    
    plt.tight_layout()
    plt.show()
else:
    print("Les variables clau especificades no es troben al dataset. Ajusta 'key_vars'.")

### 4.4 Scatter Plots (Relacions entre Variables)

In [ ]:
# Scatter plots per parelles de variables rellevants
# Exemple: edat vs MMSE, edat vs volum cerebral, etc.
scatter_pairs = [
    ('age', 'MMSE'),
    ('age', 'brain_volume'),
    ('MMSE', 'brain_volume')
]  # Ajusta segons el teu dataset

# Filtrar parelles que existeixen
valid_pairs = [(x, y) for x, y in scatter_pairs if x in df.columns and y in df.columns]

if valid_pairs:
    n_pairs = len(valid_pairs)
    fig, axes = plt.subplots(1, n_pairs, figsize=(6*n_pairs, 5))
    if n_pairs == 1:
        axes = [axes]
    
    for idx, (x_var, y_var) in enumerate(valid_pairs):
        for group in df[target_col].unique():
            mask = df[target_col] == group
            axes[idx].scatter(df.loc[mask, x_var], df.loc[mask, y_var], 
                            label=f'{target_col}={group}', alpha=0.6, s=50)
        
        axes[idx].set_xlabel(x_var, fontsize=11)
        axes[idx].set_ylabel(y_var, fontsize=11)
        axes[idx].set_title(f'{x_var} vs {y_var}', fontweight='bold', fontsize=12)
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Les parelles de variables especificades no es troben al dataset. Ajusta 'scatter_pairs'.")

### 4.5 Gràfics de Barres per Variables Categòriques

In [ ]:
# Gràfics de barres per variables categòriques segmentades per demència
for col in categorical_cols:
    if col != target_col:
        plt.figure(figsize=(10, 5))
        
        # Crear taula de contingència
        ct = pd.crosstab(df[col], df[target_col], normalize='index') * 100
        
        ct.plot(kind='bar', stacked=False, color=['#2ecc71', '#e74c3c'])
        plt.title(f'Distribució de {col} per Grup de Demència', fontweight='bold', fontsize=13)
        plt.xlabel(col, fontsize=11)
        plt.ylabel('Percentatge (%)', fontsize=11)
        plt.legend(title=target_col)
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()

## 5. Valors Anòmals o Absents

Identifiquem i analitzem valors nuls i outliers.

### 5.1 Valors Nuls

In [ ]:
# Anàlisi de valors nuls
print("Anàlisi de Valors Nuls:")
print("="*80)

missing_data = pd.DataFrame({
    'Nombre de Nuls': df.isnull().sum(),
    'Percentatge (%)': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Nombre de Nuls', ascending=False)

print(missing_data[missing_data['Nombre de Nuls'] > 0])

if missing_data['Nombre de Nuls'].sum() == 0:
    print("\n✓ No hi ha valors nuls al dataset!")
else:
    print(f"\n⚠️ Total de valors nuls: {missing_data['Nombre de Nuls'].sum()}")

In [ ]:
# Visualització de valors nuls
if missing_data['Nombre de Nuls'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_cols = missing_data[missing_data['Nombre de Nuls'] > 0]
    
    plt.barh(missing_cols.index, missing_cols['Percentatge (%)'], color='coral')
    plt.xlabel('Percentatge de Valors Nuls (%)', fontsize=12)
    plt.ylabel('Variables', fontsize=12)
    plt.title('Percentatge de Valors Nuls per Variable', fontweight='bold', fontsize=14)
    plt.grid(True, alpha=0.3, axis='x')
    
    for i, v in enumerate(missing_cols['Percentatge (%)']):
        plt.text(v + 0.5, i, f'{v:.1f}%', va='center')
    
    plt.tight_layout()
    plt.show()

### 5.2 Detecció d'Outliers (Mètode IQR)

In [ ]:
# Detecció d'outliers utilitzant el mètode IQR (Interquartile Range)
def detect_outliers_iqr(data, column):
    """
    Detecta outliers utilitzant el mètode IQR.
    Outliers són valors fora del rang [Q1 - 1.5*IQR, Q3 + 1.5*IQR]
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

print("Detecció d'Outliers (Mètode IQR):")
print("="*80)

outlier_summary = []

for col in numeric_features:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    n_outliers = len(outliers)
    pct_outliers = (n_outliers / len(df)) * 100
    
    outlier_summary.append({
        'Variable': col,
        'Nombre Outliers': n_outliers,
        'Percentatge (%)': round(pct_outliers, 2),
        'Límit Inferior': round(lower, 2),
        'Límit Superior': round(upper, 2)
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df)

total_outliers = outlier_df['Nombre Outliers'].sum()
print(f"\nTotal d'outliers detectats: {total_outliers}")

In [ ]:
# Visualització d'outliers amb box plots
n_cols = 3
n_rows = (len(numeric_features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() if n_rows > 1 else [axes]

for idx, col in enumerate(numeric_features):
    axes[idx].boxplot(df[col].dropna(), vert=True, patch_artist=True,
                     boxprops=dict(facecolor='lightblue', alpha=0.7),
                     medianprops=dict(color='red', linewidth=2))
    axes[idx].set_title(f'Box Plot: {col}', fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3, axis='y')

# Amagar eixos sobrants
for idx in range(len(numeric_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 5.3 Propostes d'Acció per Valors Anòmals

In [ ]:
print("Propostes d'Acció:")
print("="*80)

# Per valors nuls
if missing_data['Nombre de Nuls'].sum() > 0:
    print("\n📌 VALORS NULS:")
    for col in missing_data[missing_data['Nombre de Nuls'] > 0].index:
        pct = missing_data.loc[col, 'Percentatge (%)']
        if pct < 5:
            print(f"  - {col}: Eliminar files ({pct:.1f}% de dades perdudes)")
        elif pct < 30:
            if col in numeric_cols:
                print(f"  - {col}: Imputar amb mediana o mitjana ({pct:.1f}% de dades perdudes)")
            else:
                print(f"  - {col}: Imputar amb moda o categoria 'Unknown' ({pct:.1f}% de dades perdudes)")
        else:
            print(f"  - {col}: Considerar eliminar la variable ({pct:.1f}% de dades perdudes)")

# Per outliers
if total_outliers > 0:
    print("\n📌 OUTLIERS:")
    for _, row in outlier_df[outlier_df['Nombre Outliers'] > 0].iterrows():
        col = row['Variable']
        pct = row['Percentatge (%)']
        if pct < 1:
            print(f"  - {col}: Revisar manualment i considerar eliminació ({pct:.1f}%)")
        elif pct < 5:
            print(f"  - {col}: Aplicar transformació (log, sqrt) o winsorització ({pct:.1f}%)")
        else:
            print(f"  - {col}: Poden ser valors legítims, revisar context clínic ({pct:.1f}%)")

if missing_data['Nombre de Nuls'].sum() == 0 and total_outliers == 0:
    print("\n✓ No s'han detectat problemes significatius amb valors nuls o outliers.")

## 6. Anàlisi Bivariada i Correlacions

Examinem les relacions entre variables i amb la variable objectiu.

### 6.1 Matriu de Correlació

In [ ]:
# Calcular matriu de correlació
correlation_matrix = df[numeric_cols].corr()

# Visualització de la matriu de correlació
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriu de Correlació de Variables Numèriques', fontweight='bold', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

### 6.2 Correlacions amb la Variable Objectiu

In [ ]:
# Correlacions amb la variable objectiu
if target_col in numeric_cols:
    target_correlations = correlation_matrix[target_col].drop(target_col).sort_values(ascending=False)
    
    print("Correlacions amb la Variable Objectiu:")
    print("="*80)
    print(target_correlations)
    
    # Visualització
    plt.figure(figsize=(10, 6))
    colors = ['green' if x > 0 else 'red' for x in target_correlations]
    target_correlations.plot(kind='barh', color=colors, edgecolor='black')
    plt.title(f'Correlació de Variables amb {target_col}', fontweight='bold', fontsize=14)
    plt.xlabel('Coeficient de Correlació', fontsize=12)
    plt.ylabel('Variables', fontsize=12)
    plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    # Identificar correlacions fortes
    strong_corr = target_correlations[abs(target_correlations) > 0.5]
    if len(strong_corr) > 0:
        print("\n📌 Variables amb correlació FORTA (|r| > 0.5):")
        for var, corr in strong_corr.items():
            print(f"  - {var}: {corr:.3f}")
    else:
        print("\n⚠️ No s'han trobat correlacions fortes amb la variable objectiu.")
else:
    print(f"La variable objectiu '{target_col}' no és numèrica. Utilitzarem altres mètodes d'anàlisi.")

### 6.3 Detecció de Multicolinealitat

In [ ]:
# Identificar parelles de variables amb alta correlació (possibles redundàncies)
print("Detecció de Multicolinealitat:")
print("="*80)

# Crear una màscara per la meitat superior de la matriu
upper_triangle = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)

# Trobar parelles amb correlació alta
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:  # Llindar de 0.7
            high_corr_pairs.append({
                'Variable 1': correlation_matrix.columns[i],
                'Variable 2': correlation_matrix.columns[j],
                'Correlació': round(correlation_matrix.iloc[i, j], 3)
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values(by='Correlació', 
                                                              key=abs, ascending=False)
    print("\n⚠️ Parelles de variables amb ALTA correlació (|r| > 0.7):")
    print(high_corr_df.to_string(index=False))
    print("\n💡 Recomanació: Considerar eliminar una de les variables de cada parella per evitar redundància.")
else:
    print("\n✓ No s'ha detectat multicolinealitat significativa entre variables.")

### 6.4 Anàlisi de Variables Categòriques vs Objectiu

In [ ]:
# Test de Chi-quadrat per variables categòriques
from scipy.stats import chi2_contingency

print("Anàlisi d'Associació: Variables Categòriques vs Variable Objectiu")
print("="*80)

chi2_results = []

for col in categorical_cols:
    if col != target_col:
        # Crear taula de contingència
        contingency_table = pd.crosstab(df[col], df[target_col])
        
        # Test de Chi-quadrat
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        
        chi2_results.append({
            'Variable': col,
            'Chi-quadrat': round(chi2, 3),
            'p-valor': round(p_value, 4),
            'Significativa': 'Sí' if p_value < 0.05 else 'No'
        })

if chi2_results:
    chi2_df = pd.DataFrame(chi2_results).sort_values(by='p-valor')
    print(chi2_df.to_string(index=False))
    
    significant_vars = chi2_df[chi2_df['Significativa'] == 'Sí']
    if len(significant_vars) > 0:
        print(f"\n📌 {len(significant_vars)} variable(s) categòrica(es) mostren associació significativa amb {target_col} (p < 0.05)")
    else:
        print(f"\n⚠️ Cap variable categòrica mostra associació significativa amb {target_col}")
else:
    print("No hi ha variables categòriques per analitzar (a part de la variable objectiu).")

### 6.5 Comparació de Mitjanes entre Grups (T-test)

In [ ]:
# T-test per comparar mitjanes entre grups
from scipy.stats import ttest_ind

print("Comparació de Mitjanes entre Grups (T-test):")
print("="*80)

# Assumim que la variable objectiu és binària
groups = df[target_col].unique()

if len(groups) == 2:
    group1_data = df[df[target_col] == groups[0]]
    group2_data = df[df[target_col] == groups[1]]
    
    ttest_results = []
    
    for col in numeric_features:
        # Eliminar valors nuls
        g1 = group1_data[col].dropna()
        g2 = group2_data[col].dropna()
        
        if len(g1) > 0 and len(g2) > 0:
            t_stat, p_value = ttest_ind(g1, g2)
            
            ttest_results.append({
                'Variable': col,
                f'Mitjana Grup {groups[0]}': round(g1.mean(), 2),
                f'Mitjana Grup {groups[1]}': round(g2.mean(), 2),
                'Diferència': round(g2.mean() - g1.mean(), 2),
                't-statistic': round(t_stat, 3),
                'p-valor': round(p_value, 4),
                'Significativa': 'Sí' if p_value < 0.05 else 'No'
            })
    
    if ttest_results:
        ttest_df = pd.DataFrame(ttest_results).sort_values(by='p-valor')
        print(ttest_df.to_string(index=False))
        
        significant_diffs = ttest_df[ttest_df['Significativa'] == 'Sí']
        if len(significant_diffs) > 0:
            print(f"\n📌 {len(significant_diffs)} variable(s) mostren diferències significatives entre grups (p < 0.05)")
        else:
            print("\n⚠️ Cap variable mostra diferències significatives entre grups")
else:
    print(f"La variable objectiu té {len(groups)} categories. T-test només és aplicable per 2 grups.")

## 7. Resum de Descobertes

Sintetitzem les conclusions clau de l'anàlisi exploratòria.

In [ ]:
print("="*80)
print("RESUM DE DESCOBERTES CLAU")
print("="*80)

discoveries = []

# 1. Equilibri de classes
if 'imbalance_ratio' in locals():
    if imbalance_ratio > 2:
        discoveries.append(
            f"1️⃣ DESEQUILIBRI DE CLASSES: El dataset presenta un desequilibri significatiu "
            f"({imbalance_ratio:.1f}:1). Caldrà aplicar tècniques de balanceig (SMOTE, undersampling, etc.) "
            f"o utilitzar mètriques adequades (F1-score, AUC-ROC) en lloc d'accuracy."
        )
    else:
        discoveries.append(
            f"1️⃣ EQUILIBRI DE CLASSES: El dataset està relativament equilibrat ({imbalance_ratio:.1f}:1), "
            f"la qual cosa facilita l'entrenament de models."
        )

# 2. Valors nuls i qualitat de dades
total_missing = missing_data['Nombre de Nuls'].sum()
if total_missing > 0:
    pct_missing = (total_missing / (len(df) * len(df.columns))) * 100
    discoveries.append(
        f"2️⃣ QUALITAT DE DADES: S'han detectat {total_missing} valors nuls ({pct_missing:.1f}% del dataset). "
        f"Les variables més afectades són: {', '.join(missing_data[missing_data['Nombre de Nuls'] > 0].head(3).index.tolist())}. "
        f"Es recomana imputació o eliminació segons el percentatge de dades perdudes."
    )
else:
    discoveries.append(
        "2️⃣ QUALITAT DE DADES: El dataset no conté valors nuls, la qual cosa indica una bona qualitat de dades."
    )

# 3. Outliers
if 'total_outliers' in locals() and total_outliers > 0:
    pct_outliers = (total_outliers / len(df)) * 100
    top_outlier_vars = outlier_df.nlargest(3, 'Nombre Outliers')['Variable'].tolist()
    discoveries.append(
        f"3️⃣ OUTLIERS: S'han detectat {total_outliers} outliers ({pct_outliers:.1f}% de les observacions). "
        f"Les variables amb més outliers són: {', '.join(top_outlier_vars)}. "
        f"Cal revisar si són errors o valors legítims abans d'aplicar transformacions."
    )
else:
    discoveries.append(
        "3️⃣ OUTLIERS: No s'han detectat outliers significatius utilitzant el mètode IQR."
    )

# 4. Correlacions i relacions
if target_col in numeric_cols and 'strong_corr' in locals():
    if len(strong_corr) > 0:
        top_corr_var = strong_corr.abs().idxmax()
        top_corr_val = strong_corr[top_corr_var]
        discoveries.append(
            f"4️⃣ PREDICTORS RELLEVANTS: S'han identificat {len(strong_corr)} variable(s) amb correlació forta "
            f"amb la variable objectiu. La més rellevant és '{top_corr_var}' (r={top_corr_val:.3f}). "
            f"Aquestes variables seran clau per al modelatge predictiu."
        )
    else:
        discoveries.append(
            "4️⃣ PREDICTORS RELLEVANTS: No s'han trobat correlacions lineals fortes amb la variable objectiu. "
            "Pot ser necessari explorar relacions no lineals o interaccions entre variables."
        )

# 5. Multicolinealitat
if 'high_corr_pairs' in locals() and len(high_corr_pairs) > 0:
    discoveries.append(
        f"5️⃣ MULTICOLINEALITAT: S'han detectat {len(high_corr_pairs)} parella(es) de variables amb alta correlació. "
        f"Es recomana eliminar variables redundants per millorar la interpretabilitat del model i evitar problemes "
        f"en models lineals."
    )
else:
    discoveries.append(
        "5️⃣ MULTICOLINEALITAT: No s'ha detectat multicolinealitat significativa entre predictors."
    )

# Imprimir descobertes
for discovery in discoveries:
    print(f"\n{discovery}")
    print("-" * 80)

print("\n" + "="*80)
print("CONCLUSIONS I RECOMANACIONS PER AL MODELATGE")
print("="*80)
print("""
Basant-nos en l'anàlisi exploratòria realitzada, es recomana:

📋 PREPROCESSAMENT:
   • Tractar valors nuls mitjançant imputació o eliminació segons el cas
   • Revisar i tractar outliers (transformacions, winsorització o eliminació)
   • Normalitzar/estandarditzar variables numèriques si cal
   • Codificar variables categòriques (one-hot encoding, label encoding)

🎯 SELECCIÓ DE CARACTERÍSTIQUES:
   • Prioritzar variables amb correlació forta amb la variable objectiu
   • Eliminar variables redundants (alta multicolinealitat)
   • Considerar enginyeria de característiques (interaccions, transformacions)

⚖️ BALANCEIG DE CLASSES (si cal):
   • Aplicar SMOTE, ADASYN o altres tècniques de sobremuestreig
   • Considerar submuestreig de la classe majoritària
   • Utilitzar pesos de classe en l'entrenament del model

📊 VALIDACIÓ I MÈTRIQUES:
   • Utilitzar validació creuada estratificada
   • Prioritzar mètriques adequades: F1-score, AUC-ROC, Precision-Recall
   • Analitzar matriu de confusió per entendre errors del model
""")
print("="*80)

## Conclusió

Aquest notebook ha completat una anàlisi exploratòria exhaustiva del dataset de demència, cobrint:

✅ Càrrega i inspecció inicial del dataset  
✅ Anàlisi de la distribució de la variable objectiu  
✅ Estadístiques descriptives univariants  
✅ Visualitzacions comprehensives (histogrames, box plots, scatter plots)  
✅ Detecció i anàlisi de valors nuls i outliers  
✅ Anàlisi bivariada i correlacions  
✅ Síntesi de descobertes clau i recomanacions  

Els resultats d'aquesta EDA proporcionen una base sòlida per a les següents fases del projecte:
- Preprocessament de dades
- Enginyeria de característiques
- Selecció de models
- Entrenament i validació

---

**Pròxims passos suggerits:**
1. Implementar el pipeline de preprocessament basat en les descobertes
2. Crear noves característiques rellevants
3. Experimentar amb diferents algoritmes de machine learning
4. Optimitzar hiperparàmetres dels models seleccionats
5. Avaluar i comparar el rendiment dels models